## <span style="color:#ff5f27">👨🏻‍🏫 Create Deployment </span>

In this notebook, you'll create a deployment for your recommendation system.

**NOTE Currently the transformer scripts are not implemented.**

In [ ]:
import sys
from pathlib import Path

root_dir = str(Path().absolute().parent)
if root_dir not in sys.path:
    sys.path.append(root_dir)

# Exit the notebook
print("BAAAAM")
sys.exit("Exiting notebook")

In [22]:
import time

# Start the timer
notebook_start_time = time.time()

## <span style="color:#ff5f27">📝 Imports </span>

In [23]:
!pip install hopsworks


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [24]:
import os

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [25]:
import hopsworks

project = hopsworks.login()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/15551


In [26]:
# Connect to Hopsworks Model Registry
mr = project.get_model_registry()

dataset_api = project.get_dataset_api()

Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🚀 Ranking Model Deployment </span>


You start by deploying your ranking model. Since it is a CatBoost model you need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it.

In [27]:
ranking_model = mr.get_best_model(
    name="ranking_model", 
    metric="fscore", 
    direction="max",
)
ranking_model

Model(name: 'ranking_model', version: 1)

In [28]:
%%writefile ranking_transformer.py

import os
import pandas as pd
import hopsworks
import logging


class Transformer(object):
    
    def __init__(self):
        # Connect to Hopsworks
        project = hopsworks.connection().get_project()
        self.fs = project.get_feature_store()
        
        # Retrieve the 'articles' feature view
        self.articles_fv = self.fs.get_feature_view(
            name="articles", 
            version=1,
        )
        
        # Get list of feature names for articles
        self.articles_features = [feat.name for feat in self.articles_fv.schema]
        
        # Retrieve the 'customers' feature view
        self.customer_fv = self.fs.get_feature_view(
            name="customers", 
            version=1,
        )

        # Retrieve the 'candidate_embeddings' feature view
        self.candidate_index = self.fs.get_feature_view(
            name="candidate_embeddings", 
            version=1,
        )

        # Retrieve ranking model
        mr = project.get_model_registry()
        model = mr.get_model(
            name="ranking_model", 
            version=1,
        )
        
        # Extract input schema from the model
        input_schema = model.model_schema["input_schema"]["columnar_schema"]
        
        # Get the names of features expected by the ranking model
        self.ranking_model_feature_names = [feat["name"] for feat in input_schema]
            
    def preprocess(self, inputs):
        # Extract the input instance
        inputs = inputs["instances"][0]
        
        # Extract customer_id from inputs
        customer_id = inputs["customer_id"]
        
        # Search for candidate items
        neighbors = self.candidate_index.find_neighbors(
            inputs["query_emb"], 
            k=100,
        )
        neighbors = [neighbor[0] for neighbor in neighbors]
        
        # Get IDs of items already bought by the customer
        already_bought_items_ids = self.fs.sql(
            f"SELECT article_id from transactions_1 WHERE customer_id = '{customer_id}'"
        ).values.reshape(-1).tolist()
        
        # Filter candidate items to exclude those already bought by the customer
        item_id_list = [
            str(item_id) 
            for item_id 
            in neighbors 
            if str(item_id) 
            not in already_bought_items_ids
        ]
        item_id_df = pd.DataFrame({"article_id" : item_id_list})
                        
        # Retrieve Article data for candidate items
        articles_data = [
            self.articles_fv.get_feature_vector({"article_id": item_id}) 
            for item_id 
            in item_id_list
        ]
        
        logging.info(f"✅ Articles Data Retrieved!")

        articles_df = pd.DataFrame(
            data=articles_data, 
            columns=self.articles_features,
        )
        
        # Join candidate items with their features
        ranking_model_inputs = item_id_df.merge(
            articles_df, 
            on="article_id", 
            how="inner",
        )        
        
        logging.info(f"✅ Inputs are almost ready!")

        # Add customer features
        customer_features = self.customer_fv.get_feature_vector(
            {"customer_id": customer_id}, 
            return_type="pandas",
        )
        ranking_model_inputs["age"] = customer_features.age.values[0]   
        ranking_model_inputs["month_sin"] = inputs["month_sin"]
        ranking_model_inputs["month_cos"] = inputs["month_cos"]
        
        # Select only the features required by the ranking model
        ranking_model_inputs = ranking_model_inputs[self.ranking_model_feature_names]
        
        logging.info(f"✅ Inputs are ready!")
                
        return { 
            "inputs" : [{"ranking_features": ranking_model_inputs.values.tolist(), "article_ids": item_id_list}]
        }

    def postprocess(self, outputs):
        
        logging.info(f"✅ Predictions are ready!")
        
        # Extract predictions from the outputs
        preds = outputs["predictions"]
        
        # Merge prediction scores and corresponding article IDs into a list of tuples
        ranking = list(zip(preds["scores"], preds["article_ids"]))
        
        # Sort the ranking list by score in descending order
        ranking.sort(reverse=True)
        
        # Return the sorted ranking list
        return { 
            "ranking": ranking,
        }

Overwriting ranking_transformer.py


In [29]:
# Copy transformer file into Hopsworks File System 
uploaded_file_path = dataset_api.upload(
    "ranking_transformer.py",    # File name to be uploaded
    "Resources",                 # Destination directory in Hopsworks File System 
    overwrite=True,              # Overwrite the file if it already exists
) 

# Construct the path to the uploaded transformer script
transformer_script_path = os.path.join(
    "/Projects",                 # Root directory for projects in Hopsworks
    project.name,                # Name of the current project
    uploaded_file_path,          # Path to the uploaded file within the project
)

Uploading: 0.000%|          | 0/4449 elapsed<00:00 remaining<?

In [30]:
%%writefile ranking_predictor.py

import os
import joblib
import numpy as np

import logging

class Predict(object):
    
    def __init__(self):
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/ranking_model.pkl")

    def predict(self, inputs):
        
        logging.info(f"✅ Inputs: {inputs}")
        
        # Extract ranking features and article IDs from the inputs
        features = inputs[0].pop("ranking_features")
        article_ids = inputs[0].pop("article_ids")
        
        # Log the extracted features
        logging.info("predict -> " + str(features))
        
        # Log the extracted article ids
        logging.info(f'Article IDs: {article_ids}')
        
        logging.info(f"🦅 Predicting...")

        # Predict probabilities for the positive class
        scores = self.model.predict_proba(features).tolist()
        
        # Get scores of positive class
        scores = np.asarray(scores)[:,1].tolist() 

        # Return the predicted scores along with the corresponding article IDs
        return {
            "scores": scores, 
            "article_ids": article_ids,
        }

Overwriting ranking_predictor.py


In [31]:
# Upload predictor file to Hopsworks
uploaded_file_path = dataset_api.upload(
    "ranking_predictor.py", 
    "Resources",
    overwrite=True,
)

# Construct the path to the uploaded script
predictor_script_path = os.path.join(
    "/Projects",
    project.name,
    uploaded_file_path,
)

Uploading: 0.000%|          | 0/1117 elapsed<00:00 remaining<?

With that in place, you can finally deploy your model.

In [37]:
from hsml.transformer import Transformer

ranking_deployment_name = "rankingdeployment2"

# Define transformer
ranking_transformer=Transformer(
    script_file=transformer_script_path,
    resources={"num_instances": 0},
)

# Deploy ranking model
ranking_deployment = ranking_model.deploy(
    name=ranking_deployment_name,
    description="Deployment that search for item candidates and scores them based on customer metadata",
    script_file=predictor_script_path,
    resources={"num_instances": 0},
    transformer=ranking_transformer,
)

Deployment created, explore it at https://c.app.hopsworks.ai:443/p/15551/deployments/322562
Before making predictions, start the deployment by using `.start()`


In [38]:
# Start the deployment
ranking_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [39]:
# Check logs in case of failure
ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/322562

Instance name: rankingdeployment2-transformer-default-00001-deployment-6bjsmml
2024-11-07 15:15:41.325 7 root INFO [<module>():180] Loading serving script
2024-11-07 15:15:44.412 7 root INFO [__init__():117] Initializing transformer for deployment: rankingdeployment2
Connected. Call `.close()` to terminate connection gracefully.
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-RC0/share/hadoop/common/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-RC0/share/hadoop/hdfs/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2024-11-07 15:15:53.428

In [40]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates['ranking'][:k]]

In [41]:
# Define a test input example
test_ranking_input = {"instances": [{
    "customer_id": "641e6f3ef3a2d537140aaa0a06055ae328a0dddf2c2c0dd6e60eb0563c7cbba0",
    "month_sin": 1.2246467991473532e-16,
    "query_emb": [0.214135289,
     0.571055949,
     0.330709577,
     -0.225899458,
     -0.308674961,
     -0.0115124583,
     0.0730511621,
     -0.495835781,
     0.625569344,
     -0.0438038409,
     0.263472944,
     -0.58485353,
     -0.307070434,
     0.0414443575,
     -0.321789205,
     0.966559],
    "month_cos": -1.0,}]}

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates, k=3)
recommendations

['699424001', '832330004', '589825001']

In [42]:
# Check logs in case of failure
ranking_deployment.get_logs(component="transformer",tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/322562

Instance name: rankingdeployment2-transformer-default-00001-deployment-6bjsmml
2024-11-07 15:16:17.715 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.011s]
2024-11-07 15:16:17.727 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.012s]
2024-11-07 15:16:17.740 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.013s]
2024-11-07 15:16:17.753 7 opensearch INFO [log_request_success():235] POST https://rest.elastic.service.consul:9200/15551__embedding_default_project_embedding_0/_search [status:200 request:0.013s]
2024-11-07 15:16:17

## <span style="color:#ff5f27">🚀 Query Model Deployment </span>

Next, you'll deploy your query model.

In [43]:
# Retrieve the 'query_model' from the Model Registry
query_model = mr.get_model(
    name="query_model",
    version=1,
)

In [44]:
%%writefile querymodel_transformer.py

import os
import numpy as np
import pandas as pd
from datetime import datetime

import hopsworks

import logging


class Transformer(object):
    
    def __init__(self):            
        # Connect to the Hopsworks
        project = hopsworks.connection().get_project()
        ms = project.get_model_serving()
    
        # Retrieve the 'customers' feature view
        fs = project.get_feature_store()
        self.customer_fv = fs.get_feature_view(
            name="customers", 
            version=1,
        )
        # Retrieve the ranking deployment 
        self.ranking_server = ms.get_deployment("rankingdeployment")
        
        
    def preprocess(self, inputs):
        # Check if the input data contains a key named "instances"
        # and extract the actual data if present
        inputs = inputs["instances"] if "instances" in inputs else inputs
        
        # Extract customer_id and transaction_date from the inputs
        customer_id = inputs["customer_id"]
        transaction_date = inputs["transaction_date"]
        
        # Extract month from the transaction_date
        month_of_purchase = datetime.fromisoformat(inputs.pop("transaction_date"))
        
        # Get customer features
        customer_features = self.customer_fv.get_feature_vector(
            {"customer_id": customer_id}, 
            return_type="pandas",
        )
        
        # Enrich inputs with customer age
        inputs["age"] = customer_features.age.values[0]   
        
        # Calculate the sine and cosine of the month_of_purchase
        month_of_purchase = datetime.strptime(transaction_date, "%Y-%m-%dT%H:%M:%S.%f").month
        
        # Calculate a coefficient for adjusting the periodicity of the month
        coef = np.random.uniform(0, 2 * np.pi) / 12
        
        # Calculate the sine and cosine components for the month_of_purchase
        inputs["month_sin"] = float(np.sin(month_of_purchase * coef)) 
        inputs["month_cos"] = float(np.cos(month_of_purchase * coef))
                
        return {
            "instances" : [inputs]
        }
    
    def postprocess(self, outputs):
        # Return ordered ranking predictions        
        return {
            "predictions": self.ranking_server.predict({ "instances": outputs["predictions"]}),
        }

Writing querymodel_transformer.py


In [45]:
# Copy transformer file into Hopsworks File System
uploaded_file_path = dataset_api.upload(
    "querymodel_transformer.py", 
    "Models", 
    overwrite=True,
)

# Construct the path to the uploaded script
transformer_script_path = os.path.join(
    "/Projects", 
    project.name, 
    uploaded_file_path,
)

Uploading: 0.000%|          | 0/2323 elapsed<00:00 remaining<?

In [46]:
from hsml.transformer import Transformer

query_model_deployment_name = "querydeployment"

# Define transformer
query_model_transformer=Transformer(
    script_file=transformer_script_path, 
    resources={"num_instances": 0},
)

# Deploy the query model
query_model_deployment = query_model.deploy(
    name=query_model_deployment_name,
    description="Deployment that generates query embeddings from customer and item features using the query model",
    resources={"num_instances": 0},
    transformer=query_model_transformer,
)

Deployment created, explore it at https://c.app.hopsworks.ai:443/p/15551/deployments/322563
Before making predictions, start the deployment by using `.start()`


At this point, you have registered your deployment. To start it up you need to run:

In [47]:
# Start the deployment
query_model_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Deployment is already stopped


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/15551/serving/322563). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":240003,"usrMsg":"Running model deployments quota reached for Project: pbiuszti. Current: 2 Max: 2","devMsg":"Running model deployments quota reached for Project: pbiuszti. Current: 2 Max: 2","errorMsg":"Serving instance could not be started/stopped"}', error code: 240003, error msg: Serving instance could not be started/stopped, user msg: Running model deployments quota reached for Project: pbiuszti. Current: 2 Max: 2

In [21]:
# Check logs in case of failure
# query_model_deployment.get_logs(component="transformer", tail=20)

In [22]:
# Define a test input example
data = {"instances": {"customer_id": "641e6f3ef3a2d537140aaa0a06055ae328a0dddf2c2c0dd6e60eb0563c7cbba0", "transaction_date": "2022-11-15T12:16:25.330916"}}

# Test the deployment
ranked_candidates = query_model_deployment.predict(data)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates['predictions'], k=3)
recommendations

['909912001', '665118002', '581315002']

In [23]:
# Check logs in case of failure
# query_model_deployment.get_logs(component="transformer",tail=200)

Stop the deployment when you're not using it.

In [48]:
# Stop the ranking model deployment
ranking_deployment.stop()

# Stop the query model deployment
query_model_deployment.stop()

  0%|          | 0/2 [00:00<?, ?it/s]

Deployment is already stopped


---

In [25]:
# End the timer
notebook_end_time = time.time()

# Calculate and print the execution time
notebook_execution_time = notebook_end_time - notebook_start_time
print(f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds")

⌛️ Notebook Execution time: 103.20 seconds


---